In [ ]:
# Model 1: Triangular Monte Carlo Simulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from google.colab import files
import io
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

# Step 1: Upload Excel file
print("Please upload your Excel file...")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Step 2: Load data (header on row 2 = index 1)
df = pd.read_excel(io.BytesIO(uploaded[file_name]), sheet_name=0, header=1)

# Rename key columns (based on the template structure)
df.rename(columns={
    df.columns[2]: 'WBS Code',
    df.columns[3]: 'Task',
    df.columns[5]: 'Cost'  # 'Cost' is Most Likely estimate
}, inplace=True)

# Clean invalid rows and use fallback estimates
df_clean = df.copy()
df_clean = df_clean[pd.to_numeric(df_clean['Cost'], errors='coerce').notna()]
df_clean['Cost'] = pd.to_numeric(df_clean['Cost'])
df_clean['Optimistic'] = df_clean['Cost'] * 0.9
df_clean['Most Likely'] = df_clean['Cost']
df_clean['Pessimistic'] = df_clean['Cost'] * 1.2

# Step 3: Monte Carlo Simulation
NUM_SIMULATIONS = 10000
task_list = df_clean['Task'].values.tolist()
task_sim_matrix = np.zeros((NUM_SIMULATIONS, len(task_list)))

for idx, row in df_clean.iterrows():
    samples = np.random.triangular(
        row['Optimistic'], row['Most Likely'], row['Pessimistic'], NUM_SIMULATIONS
    )
    task_sim_matrix[:, idx] = samples

total_costs = task_sim_matrix.sum(axis=1)

# Step 4: Summary statistics
mean_cost = total_costs.mean()
median_cost = np.percentile(total_costs, 50)
p90_cost = np.percentile(total_costs, 90)
p10_cost = np.percentile(total_costs, 10)
std_dev = total_costs.std()

print("Monte Carlo Simulation Summary")
print(f"Mean Cost: £{mean_cost:,.2f}")
print(f"P50 (Median): £{median_cost:,.2f}")
print(f"P90: £{p90_cost:,.2f}")
print(f"P10: £{p10_cost:,.2f}")
print(f"Standard Deviation: £{std_dev:,.2f}")

# Step 5: Distribution Plot
plt.figure(figsize=(10, 6))
sns.histplot(total_costs, bins=60, kde=True, color='skyblue')
plt.axvline(mean_cost, color='red', linestyle='--', label=f'Mean: £{mean_cost:,.0f}')
plt.axvline(p90_cost, color='orange', linestyle='--', label=f'P90: £{p90_cost:,.0f}')
plt.axvline(p10_cost, color='purple', linestyle='--', label=f'P10: £{p10_cost:,.0f}')
plt.title('Monte Carlo Simulation: Total Project Cost Distribution')
plt.xlabel('Total Project Cost (£)')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()
plt.show()

# Step 6: Sensitivity Analysis (Tornado)
sensitivity_data = []
for i, task in enumerate(task_list):
    without_task = total_costs - task_sim_matrix[:, i]
    delta = total_costs.mean() - without_task.mean()
    sensitivity_data.append((task, delta))

sensitivity_df = pd.DataFrame(sensitivity_data, columns=['Task', 'Mean Contribution'])
sensitivity_df.sort_values(by='Mean Contribution', ascending=True, inplace=True)

# Tornado Chart
plt.figure(figsize=(10, 6))
sns.barplot(x='Mean Contribution', y='Task', data=sensitivity_df, palette='Blues_r')
plt.axvline(0, color='black', linewidth=0.8)
plt.title("Tornado Chart: Sensitivity of Task Cost to Total Cost")
plt.xlabel("Mean Contribution (£)")
plt.ylabel("Task")
plt.tight_layout()
plt.show()

# Step 7: Correlation Analysis
correlation_data = []
for i, task in enumerate(task_list):
    corr, _ = pearsonr(task_sim_matrix[:, i], total_costs)
    correlation_data.append((task, corr))

correlation_df = pd.DataFrame(correlation_data, columns=['Task', 'Correlation Coefficient'])
correlation_df.sort_values(by='Correlation Coefficient', ascending=True, inplace=True)

# Correlation Chart
plt.figure(figsize=(10, 6))
sns.barplot(x='Correlation Coefficient', y='Task', data=correlation_df, palette='coolwarm')
plt.axvline(0, color='black', linewidth=0.8)
plt.title("Correlation of Task Costs with Total Project Cost")
plt.xlabel("Pearson Correlation Coefficient")
plt.ylabel("Task")
plt.tight_layout()
plt.show()
